Install packages

```bash
!pip install ragas
!pip install langchain
!pip install langchain_community
!pip install langchain-openai


## 1. Fix api key

```python

import os

os.environ["OPENAI_API_KEY"] = "your-openai-key"
```

# 2. Load some data 

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.nba.com/stats/players")
documents = loader.load()

#https://www.nba.com/stats/players

In [4]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [5]:
# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

In [6]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# 3. GENERATE

In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



In [8]:
# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=3, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})


embedding nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/4 [00:00<?, ?it/s]

max retries exceeded for SimpleEvolution(generator_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=60, max_retries=15, max_wait=90, max_workers=16, exception_types=<class 'openai.RateLimitError'>)), docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x150772190>, nodes=[Node(page_content="Season Leaders | Stats | NBA.com\n\nNavigation ToggleNBAHomeTickets2023-24 Season ScheduleU.S. Radio BroadcastersLeague Pass ScheduleLivestream ScheduleKey DatesTicketsFeaturedNBA TVBracketLatestSeriesScheduleHomeTop StoriesNBA AwardsDraftTransactionsFantasyFeaturesHistoryWriter ArchiveMoreStats HomePlayersTeamsLeadersStats 101Cume StatsLineups ToolMedia Central Game StatsDraftQuick LinksContact UsAtlanticBoston CelticsBrooklyn NetsNew York KnicksPhiladelphia 76ersToronto RaptorsCentralChicago BullsCleveland CavaliersDetroit PistonsIndiana PacersMilwaukee BucksSoutheastAtlanta HawksCharlotte HornetsMiami HeatOrlando MagicWashington WizardsNorthwestD

In [9]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is Brandon Ingram's scoring average per g...,[Shai Gilgeous-AlexanderOKC31.7Anthony Edwards...,nan,simple,[{'source': 'https://www.nba.com/stats/players...,True


In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness

